In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys

In [ ]:
project_path = '/content/drive/My Drive/ML project 2/'
sys.path.append(project_path)

In [ ]:
!pip install -U segmentation-models-pytorch

In [ ]:
import os
import torchvision.transforms as T
import torch
import json
from levin.train_evaluate import TrainAndEvaluate
from levin.models.unetpp.unetpp_dataset import UnetPlusPlusRoadSegmentationDataset
from levin.models.unetpp.unetpp import UnetPlusPlus
from levin.loss_functions import ComboLoss, DiceLoss, TverskyLoss, WeightedBCEWithLogitsLoss
from torchvision.transforms.functional import InterpolationMode
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [ ]:
image_dir = os.path.join(project_path, "data", "training", "augmented", "images")
mask_dir = os.path.join(project_path, "data", "training", "augmented", "masks")

external_image_dir = os.path.join(project_path, "data", "training", "complete_data_augmented", "images")
external_mask_dir = os.path.join(project_path, "data", "training", "complete_data_augmented", "masks")

chicago_image_dir = os.path.join(project_path, "data", "training", "chicago_data_augmented", "images")
chicago_mask_dir = os.path.join(project_path, "data", "training", "chicago_data_augmented", "masks")

In [ ]:
# Load files for original augmented, external images and chicago images
image_filenames = sorted(os.listdir(image_dir))
mask_filenames = sorted(os.listdir(mask_dir))

external_image_filenames = sorted(os.listdir(external_image_dir))
external_mask_filenames = sorted(os.listdir(external_mask_dir))

chicago_image_filenames = sorted(os.listdir(chicago_image_dir))
chicago_mask_filenames = sorted(os.listdir(chicago_mask_dir))

In [ ]:
encoder_name="resnet50"
encoder_weight="imagenet"

In [ ]:
### Original augmented data
# Create train test and validation set for original augmented data
train_images, test_images, train_masks, test_masks = train_test_split(
    image_filenames, mask_filenames, test_size=0.2, random_state=42
)
train_images, validation_images, train_masks, validation_masks = train_test_split(
    train_images, train_masks, test_size=0.1, random_state=42)

print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(validation_images)}")
print(f"Number of test images: {len(test_images)}")

# Define datasets for train and test sets
train_dataset = UnetPlusPlusRoadSegmentationDataset(image_dir, mask_dir, train_images, train_masks, encoder_name, encoder_weight)
test_dataset = UnetPlusPlusRoadSegmentationDataset(image_dir, mask_dir, test_images, test_masks, encoder_name, encoder_weight)
validation_dataset = UnetPlusPlusRoadSegmentationDataset(image_dir, mask_dir, validation_images, validation_masks, encoder_name, encoder_weight)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)
validation_loader = DataLoader(validation_dataset, batch_size=8, shuffle=False, num_workers=2)


### Complete augmented original and external data
# Create train test and validation set for external data
external_train_images, external_test_images, external_train_masks, external_test_masks = train_test_split(
    external_image_filenames, external_mask_filenames, test_size=0.2, random_state=42
)
external_train_images, external_validation_images, external_train_masks, external_validation_masks = train_test_split(
    external_train_images, external_train_masks, test_size=0.1, random_state=42)

print(f"Number of external training images: {len(external_train_images)}")
print(f"Number of external validation images: {len(external_validation_images)}")
print(f"Number of external test images: {len(external_test_images)}")

# Define datasets for train and test sets
external_train_dataset = UnetPlusPlusRoadSegmentationDataset(external_image_dir, external_mask_dir, external_train_images, external_train_masks, encoder_name, encoder_weight)
external_test_dataset = UnetPlusPlusRoadSegmentationDataset(external_image_dir, external_mask_dir, external_test_images, external_test_masks, encoder_name, encoder_weight)
external_validation_dataset = UnetPlusPlusRoadSegmentationDataset(external_image_dir, external_mask_dir, external_validation_images, external_validation_masks, encoder_name, encoder_weight)

external_train_loader = DataLoader(external_train_dataset, batch_size=8, shuffle=True, num_workers=2)
external_test_loader = DataLoader(external_test_dataset, batch_size=8, shuffle=False, num_workers=2)
external_validation_loader = DataLoader(external_validation_dataset, batch_size=8, shuffle=False, num_workers=2)


### Augmented original and chicago data
# Create train test and validation set for external chicago data
chicago_train_images, chicago_test_images, chicago_train_masks, chicago_test_masks = train_test_split(
    chicago_image_filenames, chicago_mask_filenames, test_size=0.2, random_state=42
)
chicago_train_images, chicago_validation_images, chicago_train_masks, chicago_validation_masks = train_test_split(
    chicago_train_images, chicago_train_masks, test_size=0.1, random_state=42)

print(f"Number of chicago training images: {len(chicago_train_images)}")
print(f"Number of chicago validation images: {len(chicago_validation_images)}")
print(f"Number of chicago test images: {len(chicago_test_images)}")

# Define datasets for train and test sets
chicago_train_dataset = UnetPlusPlusRoadSegmentationDataset(chicago_image_dir, chicago_mask_dir, chicago_train_images, chicago_train_masks, encoder_name, encoder_weight)
chicago_test_dataset = UnetPlusPlusRoadSegmentationDataset(chicago_image_dir, chicago_mask_dir, chicago_test_images, chicago_test_masks, encoder_name, encoder_weight)
chicago_validation_dataset = UnetPlusPlusRoadSegmentationDataset(chicago_image_dir, chicago_mask_dir, chicago_validation_images, external_validation_masks, encoder_name, encoder_weight)

chicago_train_loader = DataLoader(chicago_train_dataset, batch_size=8, shuffle=True, num_workers=2)
chicago_test_loader = DataLoader(chicago_test_dataset, batch_size=8, shuffle=False, num_workers=2)
chicago_validation_loader = DataLoader(chicago_validation_dataset, batch_size=8, shuffle=False, num_workers=2)

Number of training images: 360
Number of validation images: 40
Number of test images: 100
Number of external training images: 6235
Number of external validation images: 693
Number of external test images: 1732
Number of chicago training images: 2005
Number of chicago validation images: 223
Number of chicago test images: 557


In [ ]:
# Test different loss functions for original augmented data
root_path = os.path.join(project_path, "levin")
save_path = os.path.join(root_path, "trained_models", "unet++")
print(f"Created save path {save_path}")
os.makedirs(save_path, exist_ok=True)

loss_functions = [torch.nn.BCEWithLogitsLoss(), WeightedBCEWithLogitsLoss(), DiceLoss(), ComboLoss(), TverskyLoss()]
best_f1 = 0
best_threshold = 0
best_epoch = 0
best_loss_function = None
best_data = "original"
for criterion in loss_functions:
  print(f"Training model with {criterion.__class__.__name__}")
  # Train and evaluate
  model = UnetPlusPlus(encoder_name=encoder_name, encoder_weight=encoder_weight, in_channels=3, num_labels=1)
  train_eval = TrainAndEvaluate(model, train_loader, validation_loader, test_loader, criterion, 10, save_path, save=False)
  f1, threshold, epoch = train_eval.run()
  if f1 > best_f1:
    best_f1 = f1
    best_threshold = threshold
    best_epoch = epoch
    best_loss_function = criterion

print(f"Best F1 score: {best_f1}")
print(f"Best threshold: {best_threshold}")
print(f"Best epoch: {best_epoch}")
print(f"Best loss function: {best_loss_function.__class__.__name__}")

Created save path /content/drive/My Drive/ML project 2/levin/trained_models/unet++
Training model with BCEWithLogitsLoss
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch1.pt
Epoch 1/10, Training Loss: 0.4876, Validation Loss: 0.3899
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch2.pt
Epoch 2/10, Training Loss: 0.2249, Validation Loss: 0.1752
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch3.pt
Epoch 3/10, Training Loss: 0.1527, Validation Loss: 0.1528
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch4.pt
Epoch 4/10, Training Loss: 0.1218, Validation Loss: 0.1709
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch5.pt
Epoch 5/10, Training Loss: 0.1070

/content/drive/My Drive/ML project 2/levin/train_evaluate.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.model.load_state_dict(torch.load(best_model_path))

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch7.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/BCEWithLogitsLoss/trained_model_epoch8.pt
Deleted model: /

/content/drive/My Drive/ML project 2/levin/train_evaluate.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.model.load_state_dict(torch.load(best_model_path))

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/unet++/WeightedBCEWithLogitsLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/WeightedBCEWithLogitsLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/WeightedBCEWithLogitsLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/WeightedBCEWithLogitsLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/WeightedBCEWithLogitsLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/WeightedBCEWithLogitsLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/WeightedBCEWithLogitsLoss/trained_model_epoch7.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/We

...

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/unet++/DiceLoss/trained_model_epoch9.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/DiceLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/DiceLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/DiceLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/DiceLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/DiceLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/DiceLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/DiceLoss/trained_model_epoch7.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/DiceLoss

/content/drive/My Drive/ML project 2/levin/train_evaluate.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.model.load_state_dict(torch.load(best_model_path))

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/trained_model_epoch9.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/trained_model_epoch7.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/

/content/drive/My Drive/ML project 2/levin/train_evaluate.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.model.load_state_dict(torch.load(best_model_path))

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/unet++/TverskyLoss/trained_model_epoch10.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/TverskyLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/TverskyLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/TverskyLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/TverskyLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/TverskyLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/TverskyLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/TverskyLoss/trained_model_epoch7.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/train

In [ ]:
# Free up memory
torch.cuda.empty_cache()

In [ ]:
best_f1 = 0.876747
best_threshold = 0.25
best_epoch = 9
best_loss_function = ComboLoss()
best_data = "original"

In [ ]:
# Train and evaluate the model with the best loss function on external data
external_save_path = os.path.join(root_path, "trained_models", "unet++", "external")
print("Created external save path")
os.makedirs(external_save_path, exist_ok=True)
model = UnetPlusPlus(encoder_name=encoder_name, encoder_weight=encoder_weight, in_channels=3, num_labels=1)
train_eval = TrainAndEvaluate(model, external_train_loader, external_validation_loader, external_test_loader, best_loss_function, 10, external_save_path)
f1, threshold, epoch = train_eval.run()
if f1 > best_f1:
    best_f1 = f1
    best_threshold = threshold
    best_epoch = epoch
    best_data = "external"

Created external save path


KeyboardInterrupt: 

In [ ]:
# Train and evaluate the model with the best loss function on chicago data
chicago_save_path = os.path.join(root_path, "trained_models", "unet++", "chicago")
print("Created chicago save path")
os.makedirs(chicago_save_path, exist_ok=True)
model = UnetPlusPlus(encoder_name=encoder_name, encoder_weight=encoder_weight, in_channels=3, num_labels=1)
train_eval = TrainAndEvaluate(model, external_train_loader, external_validation_loader, external_test_loader, best_loss_function, 10, chicago_save_path)
f1, threshold, epoch = train_eval.run()
if f1 > best_f1:
    best_f1 = f1
    best_threshold = threshold
    best_epoch = epoch
    best_data = "chicago"

Created chicago save path
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch1.pt
Epoch 1/10, Training Loss: 0.3669, Validation Loss: 0.2606
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch2.pt
Epoch 2/10, Training Loss: 0.2535, Validation Loss: 0.2446
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch3.pt
Epoch 3/10, Training Loss: 0.2244, Validation Loss: 0.2256
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch4.pt
Epoch 4/10, Training Loss: 0.2133, Validation Loss: 0.2287
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch5.pt
Epoch 5/10, Training Loss: 0.1982, Validation Loss: 0.2161
Model saved to /content/drive/My Drive/ML project 2/levin/trained_mod

/content/drive/My Drive/ML project 2/levin/train_evaluate.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.model.load_state_dict(torch.load(best_model_path))

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch10.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/chicago/ComboLoss/trained_model_epoch7.pt
Deleted model: 